<img width="8%" alt="Google Sheets.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Sheets.png" style="border-radius: 15%">

# Google Sheets - Update people database

**Tags:** #googlesheets #gsheet #data #naas_drivers #growth #leads #openai #linkedin #people

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook updates your people database with new people that interacted with content and enrich it with ICP and company.

## Input

### Import libraries

In [1]:
import naas_data_product
import naas
from naas_drivers import gsheet, linkedin
import pandas as pd
import os
from datetime import date
import openai
import time
import re

🕣 Your Production Timezone is Europe/Paris

✅ utils file '/home/ftp/abi/utils/data.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/llm.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_chat_plugin.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_lab.ipynb' successfully loaded.


### Setup variables
**Inputs**
- `input_dir`: Input directory to retrieve file from.
- `file_interactions`: Name of the file to be retrieved.
- `api_key`: LLM API Key.
- `li_at`: Cookie used to authenticate Members and API clients.
- `JSESSIONID`: Cookie used for Cross Site Request Forgery (CSRF) protection and URL signature validation.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `people_sheetname`: Google Sheets sheet name storing leads profiles.
- `prompt_seniority`: Prompt to be used to find people seniority.
- `prompt_department`: Prompt to be used to find people department.
- `prompt_company`: Prompt to be used to find people company.

**Outputs**
- `output_dir`: Output directory to save file to.
- `file_people`: Output file name to save as picke.
- `datalake_dir`: Datalake directory (outputs folder from abi project).

In [2]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "growth-engine", date.today().isoformat())
file_interactions = "linkedin_interactions"
api_key = naas.secret.get("OPENAI_API_KEY")
li_at = naas.secret.get("LINKEDIN_LI_AT") or "YOUR_LINKEDIN_LI_AT" #example: AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID") or "YOUR_LINKEDIN_JSESSIONID" #example: ajax:8379907400220387585
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
people_sheetname = "PEOPLE"
prompt_seniority = """
Find the seniority that matches the most with the occupation extracted from a LinkedIn profile, if there is no exact match, you WON'T return a sentence or ask for more information but stricly return "Professional/Staff".
Seniority definition:
- "Entry-Level": Any occupation with Intern/Internship, Trainee, Junior
- "Professional/Staff": [Role] Specialist, [Role] Analyst, [Role] Coordinator.
- "Senior Professional/Staff": Senior [Role] Specialist, Senior [Role] Analyst.
- "Lead/Supervisor": Team Lead, Supervisor.
- "Manager": Manager, [Department] Manager.
- "Senior Manager": Senior Manager, Director.
- "Executive": Vice President, Chief [Role] Officer (CFO, CTO, etc.).
- "Top Executive": President, CEO, Managing Director.
"""
prompt_department = """
Find the department that matches the most with the occupation extracted from a LinkedIn Profile, if there is no exact match, you WON'T return a sentence or ask for more information but stricly return "Not Found".
Use the list below as a starting point to understand the department affiliations.
Remember, roles can vary across industries, and some individuals may wear multiple hats in smaller companies.
Departments: 
- "Human Resources (HR)": handling employee relations, benefits, recruitment, training, and workplace culture. Keywords: "Recruiter," "HR Manager," or "Training Coordinator."
- "Finance": handling company finances, budgets, payroll, taxes, financial reporting, and investment strategies. Keywords: "Financial Analyst," "Accountant," or "Chief Financial Officer (CFO)" 
- "Marketing": handling promotion of the business, market research, developing marketing strategies, and managing advertising. Look for titles like "Marketing Director," "Brand Manager," or "Content Strategist."
- "Sales": handling revenue generation through sales strategies, customer relationships, and managing sales teams. Titles might include "Sales Representative," "Account Executive," or "Sales Manager."
- "Operations": handling day-to-day operations, production efficiency, quality control, and supply chain management. Look for "Operations Manager," "Production Supervisor," or "Supply Chain Analyst."
- "Information Technology (IT)": managing technology infrastructure, supporting systems, ensuring cybersecurity. Titles such as "IT Support Specialist," "Systems Administrator," or "Chief Information Officer (CIO)" are indicative.
- "Research and Development (R&D)": involving the development of new products or services and market research. Look for "Research Scientist," "Product Developer," or "R&D Manager."
- "Customer Service": assisting customers, maintaining satisfaction, and managing feedback. Common titles include "Customer Service Representative," "Support Technician," or "Client Relations Manager."
- "Legal": Occupations handling legal matters, compliance, contracts, and advising on legal risks. Titles like "Corporate Lawyer," "Legal Assistant," or "Compliance Officer" are relevant.
- "Procurement": look for roles related to acquiring goods and services, negotiating with suppliers. Titles might include "Procurement Officer," "Purchasing Manager," or "Supply Coordinator."
- "Quality Assurance (QA)": ensuring products and services meet standards and regulations. Look for "Quality Assurance Specialist," "QA Tester," or "Quality Manager."
- "Logistics and Supply Chain": managing the flow of goods, optimizing delivery routes, and inventory. Titles such as "Logistics Coordinator," "Supply Chain Analyst," or "Fleet Manager" are common.
- "Public Relations (PR)":  managing the company's public image, press releases, and media communications. Look for "Public Relations Specialist," "Communications Director," or "Media Coordinator."
- "Executive Management": high-level decision-makers setting company strategy. Titles include "Chief Executive Officer (CEO)," "Managing Director," or "Executive Vice President."
- "Product Management": overseeing product development and lifecycle. Titles might be "Product Manager," "Product Owner," or "Product Development Lead."
- "Strategy and Business Development": Look for roles identifying new business opportunities and planning. Titles include "Business Development Manager," "Strategic Planner," or "Growth Analyst."
- "Education": Look for roles that try to teach or educate people.
"""
prompt_company = """
I will give you the occupation from a profile I get from LinkedIn, you will return the company you can extract from by checking the word after 'at' or '@'.
If you don't find it return "NA"
Don't put the results into quotes.
"""

# Outputs
output_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "growth-engine", date.today().isoformat())
file_people = "people"
datalake_dir = os.path.join("/", "home", "ftp", "abi", "outputs")

## Model

### Get people

In [3]:
df_init = gsheet.connect(spreadsheet_url).get(sheet_name=people_sheetname)
if not isinstance(df_init, pd.DataFrame):
    df_init = pd.DataFrame()
print("- People (init):", len(df_init))
# df_leads.head(3)

- People (init): 193


### Get interactions

In [4]:
df_interactions = pload(input_dir, file_interactions)    
print('- Interactions:', len(df_interactions))
# df_interactions.head(3)

- Interactions: 291


### Extract profiles from interactions
This function will extract unique profile from interactions database with meta data.

In [5]:
def get_date_interaction(
    df_init,
    col_date,
    new_col_date,
):
    # Init
    df = df_init.copy()
    df = df.sort_values(col_date, ascending=True)
    
    # Drop duplicates
    to_keep = [
        "PROFILE_URL",
        col_date,
    ]
    df = df[to_keep].drop_duplicates(["PROFILE_URL"], keep="first").rename(columns={col_date: new_col_date})
    return df.reset_index(drop=True)

def get_interactions_by_profile(
    df_init,
    contacts
):
    # Init
    df = df_init.copy()
    interactions = {}
    
    # Cleaning
    to_select = [
        "PROFILE_URL",
        "FULLNAME",
        "CONTENT_TITLE",
        "CONTENT_URL",
        "INTERACTION",
        "INTERACTION_CONTENT",
        "DATE_INTERACTION"
    ]
    df = df[to_select].sort_values(by="PROFILE_URL").reset_index(drop=True)
    df["INTERACTION_TEXT"] = ""
    df.loc[df["INTERACTION"] == "POST_REACTION", "INTERACTION_TEXT"] = df["FULLNAME"]  + " sent '" + df["INTERACTION_CONTENT"].str.lower() + "' reaction to '" + df["CONTENT_TITLE"].str.strip() + "' (" + df["CONTENT_URL"] + ") on " + df["DATE_INTERACTION"].astype(str)
    df.loc[df["INTERACTION"] == "POST_COMMENT", "INTERACTION_TEXT"] = df["FULLNAME"]  + " commented '" + df["INTERACTION_CONTENT"].str.capitalize() + "' on '" + df["CONTENT_TITLE"].str.strip() + "' (" + df["CONTENT_URL"] + ") on " + df["DATE_INTERACTION"].astype(str)

    # Create interactions by profile
    for contact in contacts:
        tmp_df = df.copy()
        tmp_df = tmp_df[tmp_df["PROFILE_URL"] == contact].reset_index(drop=True)
        interests = []
        for row in tmp_df.itertuples():
            interaction_text = row.INTERACTION_TEXT
            interests.append(interaction_text)
        interactions[contact] = interests
    return interactions

def create_db_people(
    df_init,
    df_interactions,
    output_dir,
):
    # Init - Filter on profile
    df_people = df_interactions[df_interactions["PROFILE_URL"].str.contains("https://www.linkedin.com/in/.+")]
    df_direct = df_people.copy()
    df_score = df_people.copy()
    
    # Get first interaction -> Created date AND MQL date
    df_created = get_date_interaction(
        df_people,
        "DATE_INTERACTION",
        "CREATED_DATE",
    )
    df_created["MQL_DATE"] = df_created["CREATED_DATE"]
    
    # Get profile information with last content interaction -> Last interaction date
    to_keep = [
        "PROFILE_URL",
        "FIRSTNAME",
        "LASTNAME",
        "OCCUPATION",
        "PUBLIC_ID",
        "DATE_INTERACTION",
        "CONTENT_URL",
        "CONTENT_TITLE"
    ]
    df_direct = df_direct[to_keep].drop_duplicates(["PROFILE_URL"])
   
    # Get interactions score by profile
    df_score = df_score.groupby(["PROFILE_URL"], as_index=False).agg({"INTERACTION_SCORE": "sum"})
    
    # Merge dfs
    df = pd.merge(df_created, df_direct, how="left").fillna("NA")
    df = pd.merge(df, df_score, how="left")
    
    # Get more than 3 interactions date -> SQL date
    df_sql = get_date_interaction(
        df[df["INTERACTION_SCORE"] >= 3],
        "DATE_INTERACTION",
        "SQL_DATE",
    )
    
    # Merge dfs and cleaning: Rename columns
    to_rename = {
        "DATE_INTERACTION": "LAST_INTERACTION_DATE",
        "CONTENT_URL": "LAST_CONTENT_URL_INTERACTION",
        "CONTENT_TITLE": "LAST_CONTENT_TITLE_INTERACTION"
    }
    df = pd.merge(df, df_sql, how="left").rename(columns=to_rename).fillna("NA")

    # Cleaning: Remove emojis from name and occupation
    df["FIRSTNAME"] = df.apply(lambda row: remove_emojis(str(row["FIRSTNAME"])), axis=1)
    df["LASTNAME"] = df.apply(lambda row: remove_emojis(str(row["LASTNAME"])), axis=1)
    df["OCCUPATION"] = df.apply(lambda row: remove_emojis(str(row["OCCUPATION"])), axis=1)
    df["FULLNAME"] = df["FIRSTNAME"] + " " + df["LASTNAME"]
    
    # Create notes from interactions
    leads = df["PROFILE_URL"].unique()  
    df["NOTES"] = df["PROFILE_URL"].map(get_interactions_by_profile(df_people, leads))
    
    # Cleaning: Sort values
    df = df.sort_values(by=["INTERACTION_SCORE", "LAST_INTERACTION_DATE"], ascending=[False, False])
    
    # Get meta data from existing people
    col_ref = [
        "PROFILE_URL",
        "COMPANY",
        "SENIORITY",
        "DEPARTMENT",
        "CRM_CONTACT_ID",
    ]
    for c in col_ref:
        # If columns does not exist, init value to be determined (TBD)
        if not c in df_init.columns:
            df_init[c] = "TBD"
    ref = df_init[col_ref]

    # Merge to get meta data
    df = pd.merge(df, ref, how="left").fillna("TBD")
        
    # Cleaning
    to_order = [
        "CREATED_DATE",
        "FIRSTNAME",
        "LASTNAME",
        "FULLNAME",
        "OCCUPATION",
        "SENIORITY",
        "DEPARTMENT",
        "COMPANY",
        "INTERACTION_SCORE",
        "MQL_DATE",
        "SQL_DATE",
        "LAST_INTERACTION_DATE",
        "NOTES",
        "PROFILE_URL",
        "PUBLIC_ID",
        "LAST_CONTENT_TITLE_INTERACTION",
        "LAST_CONTENT_URL_INTERACTION",
        "CRM_CONTACT_ID"
    ]
    df = df[to_order]

    # Save database
    pdump(output_dir, df, "people_init")
    return df.reset_index(drop=True)

db_people = create_db_people(
    df_init,
    df_interactions,  
    output_dir,
)
print("- People:", len(db_people))
db_people.head(3)

- People: 193


,CREATED_DATE,FIRSTNAME,LASTNAME,FULLNAME,OCCUPATION,SENIORITY,DEPARTMENT,COMPANY,INTERACTION_SCORE,MQL_DATE,SQL_DATE,LAST_INTERACTION_DATE,NOTES,PROFILE_URL,PUBLIC_ID,LAST_CONTENT_TITLE_INTERACTION,LAST_CONTENT_URL_INTERACTION,CRM_CONTACT_ID
0,2023-12-12 17:30:06+0100,Vin,Vashishta,Vin Vashishta,AI Advisor | Author “From Data To Profit” | Co...,Professional/Staff,Strategy and Business Development,V Squared,12,2023-12-12 17:30:06+0100,2024-01-06 20:32:08+0100,2024-01-06 20:32:08+0100,[Vin Vashishta sent 'like' reaction to 'Your a...,https://www.linkedin.com/in/ACoAAADS0WQBhQQVMD...,vineetvashishta,"Without efficient workflows, hiring more peopl...",https://www.linkedin.com/feed/update/urn:li:ac...,TBD
1,2023-12-16 22:59:09+0100,Matteo,Castiello,Matteo Castiello,Generative AI Advisor and Researcher | Guiding...,Professional/Staff,Research and Development (R&D),33A,10,2023-12-16 22:59:09+0100,2024-01-06 20:32:08+0100,2024-01-06 20:32:08+0100,[Matteo Castiello sent 'praise' reaction to 'I...,https://www.linkedin.com/in/ACoAACenYg8BoVOSWA...,matteocastiello,"Without efficient workflows, hiring more peopl...",https://www.linkedin.com/feed/update/urn:li:ac...,TBD
2,2024-01-02 23:10:00+0100,Akmel,Syed,Akmel Syed,I help companies get started with AI | AI Cons...,Professional/Staff,Information Technology (IT),Qlik,8,2024-01-02 23:10:00+0100,2024-01-06 22:40:28+0100,2024-01-06 22:40:28+0100,[Akmel Syed sent 'like' reaction to 'Without e...,https://www.linkedin.com/in/ACoAAAZMWqMBrP6avP...,akmel-syed,"Without efficient workflows, hiring more peopl...",https://www.linkedin.com/feed/update/urn:li:ac...,TBD


### Enrich people with company, seniority, department

In [6]:
def enrich_from_occupation(
    occupation,
    key,
    keys,
    api_key,
    prompt,
    file,
    output_dir,
):
    result = "NA"
    if key not in keys:
        result = create_chat_completion(api_key, prompt, occupation).replace("'", "").replace('"', '')
        keys[key] = result
    else:
        result = keys.get(key)
    pdump(output_dir, keys, file)
    return result

def get_dict(
    df,
    column_name,
    key,
    file,
    output_dir
):
    data = {}
    if column_name in df.columns:
        data = pload(output_dir, file)
        if data is None: 
            data = df[~df[column_name].isin(["TBD", "NA"])].set_index(key)[column_name].to_dict()
            pdump(output_dir, data, file)
    return data

def enrich_people(
    df_init,
    prompt_seniority,
    prompt_department,
    prompt_company,
    output_dir,
    datalake_dir,
    limit_linkedin=30
):
    # Init
    df = df_init.copy()

    # Get people seniority
    people_seniority = get_dict(df, "SENIORITY", "PROFILE_URL", "people_seniority", output_dir)
    
    # Get people department
    people_department = get_dict(df, "DEPARTMENT", "PROFILE_URL", "people_department", output_dir)

    # Get companies
    people_companies = get_dict(df, "COMPANY", "PROFILE_URL", "people_companies", output_dir)
    
    # Loop on profile
    call_linkedin = 0    
    for row in df.itertuples():
        index = row.Index
        fullname = row.FULLNAME
        occupation = row.OCCUPATION
        profile_url = row.PROFILE_URL
        seniority = row.SENIORITY
        department = row.DEPARTMENT
        company = row.COMPANY
        interaction_score = row.INTERACTION_SCORE
        
        # Update ICP and Company from OpenAI
        if seniority == "TBD" or department == "TBD" or company == "TBD" and api_key != "NA":
            print(f"🤖 Finding seniority, departement & company for '{fullname}' ({profile_url}) from occupation: {occupation}")
            seniority = enrich_from_occupation(
                occupation,
                profile_url,
                people_seniority,
                api_key,
                prompt_seniority,
                "people_seniority",
                output_dir
            )
            department = enrich_from_occupation(
                occupation,
                profile_url,
                people_department,
                api_key,
                prompt_department,
                "people_department",
                output_dir
            )
            company = enrich_from_occupation(
                occupation,
                profile_url,
                people_companies,
                api_key,
                prompt_company,
                "people_companies",
                output_dir
            )
            df.loc[index, "SENIORITY"] = seniority.strip()
            df.loc[index, "DEPARTMENT"] = department.strip()
            df.loc[index, "COMPANY"] = company.strip()
            print("- Seniority:", seniority)
            print("- Department:", department)
            print("- Company:", company)
            print()
            
        # Update Company info
        if company == "NA" and interaction_score >= 3 and call_linkedin < limit_linkedin:
            print(f"🕸️ Finding LinkedIn company for '{fullname}' ({profile_url}) (interaction score: {interaction_score})")
            company_name = "Not Found"
            linkedin_dir = os.path.join(datalake_dir, "datalake", "linkedin", "profiles")
            linkedin_id = profile_url.split("/")[-1]
            tmp_df = pload(linkedin_dir, f"{linkedin_id}_linkedin_top_card")
            if tmp_df is None:
                # Get Top Card
                try:
                    tmp_df = linkedin.connect(li_at, JSESSIONID).profile.get_top_card(profile_url)
                    pdump(linkedin_dir, tmp_df, f"{linkedin_id}_linkedin_top_card")
                    time.sleep(2)
                    call_linkedin += 1
                except Exception as e:
                    print(e)
                    company_name = "ERROR_LINKEDIN_ENRICHMENT"
                    tmp_df = pd.DataFrame()
            if len(tmp_df) > 0:
                company_name = tmp_df.loc[0, "COMPANY_NAME"]
            df.loc[index, "COMPANY"] = str(company_name).replace("None", "UNKNOWN").replace("NA", "UNKNOWN").strip()
            print("- Company:", company_name)
            if call_linkedin >= limit_linkedin:
                print("🛑 Call LinkedIn reached:", limit_linkedin)
            else:
                print("- ⚠️ LinkedIn call:", call_linkedin)
            print()
    return df.reset_index(drop=True)
    
df_people = enrich_people(
    db_people,
    prompt_seniority,
    prompt_department,
    prompt_company,
    output_dir,
    datalake_dir,
)  

## Output

### Save data

In [7]:
pdump(output_dir, df_people, file_people)

### Send "People" to spreadsheet

In [8]:
df_check = pd.concat([df_init.astype(str), df_people.astype(str)]).drop_duplicates(keep=False)
if len(df_check) > 0:
    gsheet.connect(spreadsheet_url).send(data=df_people, sheet_name=people_sheetname, append=False)
else:
    print("Noting to update in Google Sheets!")    

Noting to update in Google Sheets!
